# メタデータを検証する

研究データに付与されているメタデータを検証し、その結果をレポートします。  
検証を行うことで、適切なメタデータが研究データに付与されているかを確認することができます。  

検証結果は必ず同期する必要はありません。この環境で検証結果を確認いただいたあとに、GIN-forkへ同期するか破棄するか選択いただけます。  
検証結果を同期する場合は、研究リポジトリの直下の**validation_results**フォルダに格納されます。タスク実行後に検証結果を確認する際はvalidation_resultsフォルダ配下の**results.json**をご確認ください。  
同期しない場合は、検証結果は破棄されます。

## 0. 研究リポジトリ名を確認する

以下のセルを実行すると、この実験実行環境で操作する研究リポジトリ名を確認できます。 

In [ ]:
import os
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils
utils.show_name('green')

## 1. 研究リポジトリのメタデータを用意する

メタデータを検証するための準備として、研究リポジトリのメタデータの取得と整形を行います。

In [ ]:
import os
from urllib import parse
import requests
import json
import traceback
import logging

try:
    # get current branch name
    os.chdir(os.environ['HOME'])
    result = !git branch --contains
    current_branch = result[0].replace('* ', '')

    # get repository id
    from WORKFLOWS.utils.repository_id import repository_id
    repo_id = repository_id.get_repo_id()

    # get token for ginfork
    from WORKFLOWS.utils.token import token
    ginfork_token = token.get_ginfork_token()

    # generate url_for_request_metadata
    path_flow_root = '/home/jovyan/WORKFLOWS/FLOW/'
    os.chdir(path_flow_root)
    from util.scripts import utils
    params = {}
    with open(utils.fetch_param_file_path(), mode='r') as f:
        params = json.load(f)
    parse_result = parse.urlparse(params['siblings']['ginHttp'])
    url_for_request_metadata = parse.urlunparse((
        parse_result.scheme, 
        parse_result.netloc, 
        'api/v1/repos/' + repo_id + '/' + current_branch + '/metadata',
        '',
        'token=' + ginfork_token,
        ''
    ))

    # get raw metadata from ginfork & package
    response = requests.get(url_for_request_metadata)
    result = response.json()
except:
    print('GIN-forkへの研究リポジトリのメタデータの問い合わせに失敗しました。担当者にお問い合わせください。')
    print(traceback.format_exc())
else:
    if response.status_code == 200:
        print('正常にメタデータを取得できました。')
        raw_metadata = result['AllMetadata']
        # packaging
        try:
            from dg_packager.ro_generator import gin_ro_generator
            RoGenerator = gin_ro_generator.RoGenerator(raw_metadata)
            ro_crate = RoGenerator.generate()
        except Exception as e:
            print('メタデータに不備があります。ご確認ください。')
            logging.error(str(e))
    elif response.status_code == 400:
        print('無効なリクエストです。担当者にお問い合わせください。')
        print(result['GeneralError']['message'])
    elif response.status_code == 500:
        print('システムエラーが発生しました。担当者にお問い合わせください。')
        print(result['GeneralError']['message'])

### TEST

In [ ]:
raw_metadata = {
    "research_project": {
        "name": "readsfdfd",
        "description": "afdafsadsfds"
    },
    "funder_orgs": [],
    "research_orgs": [
        {
            "@id": "https://ror.org/05wks2t16",
            "name": "University JJYSCUVI",
            "description": "MUSTSAWDGDIODSYCGSNOANLJBYASPBBOMNSMXHTBCYIEXHGRCJ",
            "alias": "MOCATQBE"
        }
    ],
    "licenses": [],
    "data_downloads": [],
    "repository_objs": [],
    "hosting_institutions": [],
    "persons": [
        {
            "id": "12",
            "url": "https://orcid.org/0000-0001-5473-8797",
            "name": "デモデモ デモ",
            "alias": "D. demo",
            "affiliation": "https://ror.org/05wks2t16",
            "email": "demo-u1@asd.com",
            "telephone": "",
            "eradResearcherNumber": ""
        }
    ],
    "files": [
        {
            "@id": ".repository_id",
            "name": ".repository_id",
            "contentSize": "2",
            "encodingFormat": "text/plain",
            "sha256": "25fc0e7096fc653718202dc30b0c580b8ab87eac11a700cba03a7c021bc35b0c",
            "url": "http://dg01.dg.rcos.nii.ac.jp/demo_user1/repo1/src/master/.repository_id",
            "sdDatePublished": "",
            "experimentPackageFlag": False
        },
        {
            "@id": "Dockerfile",
            "name": "Dockerfile",
            "contentSize": "2877",
            "encodingFormat": "text/plain",
            "sha256": "901e166e91018d368d9c851bf5cd5db7fa0671db9c99ca3a877a0534956ff64f",
            "url": "http://dg01.dg.rcos.nii.ac.jp/demo_user1/repo1/src/master/Dockerfile",
            "sdDatePublished": "",
            "experimentPackageFlag": False
        },
        {
            "@id": "LICENSE",
            "name": "LICENSE",
            "contentSize": "10261",
            "encodingFormat": "text/plain",
            "sha256": "940db9a2aedc7a188f9b6efbe1ddb0746a7f5fd2387c6ea19c98006f51a56f55",
            "url": "http://dg01.dg.rcos.nii.ac.jp/demo_user1/repo1/src/master/LICENSE",
            "sdDatePublished": "",
            "experimentPackageFlag": False
        },
        {
            "@id": "README.md",
            "name": "README.md",
            "contentSize": "3163",
            "encodingFormat": "text/plain",
            "sha256": "2d86ec3ddd9cac01ace925474f8bbb2096189192a16004d6ae79db4388741af7",
            "url": "http://dg01.dg.rcos.nii.ac.jp/demo_user1/repo1/src/master/README.md",
            "sdDatePublished": "",
            "experimentPackageFlag": False
        },
        {
            "@id": "dmp.json",
            "name": "dmp.json",
            "contentSize": "1306",
            "encodingFormat": "text/plain",
            "sha256": "384414f6f6b026d706f9622b5233e6b229532fdd90e47b4184ec2ca4bef726ab",
            "url": "http://dg01.dg.rcos.nii.ac.jp/demo_user1/repo1/src/master/dmp.json",
            "sdDatePublished": "",
            "experimentPackageFlag": False
        },
        {
            "@id": "images/maDMP_to_workflow.jpg",
            "name": "maDMP_to_workflow.jpg",
            "contentSize": "106997",
            "encodingFormat": "image/jpeg",
            "sha256": "bbf5c109de04f61d6eec1afecac646a39e1f45a1f760b861d26fffcb1980497e",
            "url": "http://dg01.dg.rcos.nii.ac.jp/demo_user1/repo1/src/master/images/maDMP_to_workflow.jpg",
            "sdDatePublished": "",
            "experimentPackageFlag": False
        },
        {
            "@id": "images/user_custom_area.png",
            "name": "user_custom_area.png",
            "contentSize": "5752",
            "encodingFormat": "image/png",
            "sha256": "207913bb1211ab6df141daca5404208f6854f012743c31f7e496f6a213409fac",
            "url": "http://dg01.dg.rcos.nii.ac.jp/demo_user1/repo1/src/master/images/user_custom_area.png",
            "sdDatePublished": "",
            "experimentPackageFlag": False
        },
        {
            "@id": "maDMP.ipynb",
            "name": "maDMP.ipynb",
            "contentSize": "8365",
            "encodingFormat": "text/plain",
            "sha256": "63271872f640787d0bbc4463a1b0b415d598635eecea43d87ce0a6dbc9305ddb",
            "url": "http://dg01.dg.rcos.nii.ac.jp/demo_user1/repo1/src/master/maDMP.ipynb",
            "sdDatePublished": "",
            "experimentPackageFlag": False
        }
    ],
    "datasets": [
        {
            "@id": "images/",
            "name": "images",
            "url": "http://dg01.dg.rcos.nii.ac.jp/demo_user1/repo1/src/master/images/"
        }
    ],
    "gin_monitoring": {
        "contentSize": "1GB",
        "workflowIdentifier": "basic",
        "datasetStructure": "with_code"
    },
    "dmps": []
}

In [ ]:
import logging
try:
    from dg_packager.ro_generator import gin_ro_generator
    RoGenerator = gin_ro_generator.RoGenerator(raw_metadata)
    ro_crate = RoGenerator.generate()
except Exception as e:
    print('メタデータに不備があります。ご確認ください。')
    logging.error(str(e))

## 2. メタデータを検証する

メタデータの検証と検証結果の生成を行う検証サービスに、メタデータの検証を依頼します。

In [ ]:
from urllib import parse
import requests
import os
import traceback
import json

try:
    # generate url_for_request_validation
    path_flow_root = '/home/jovyan/WORKFLOWS/FLOW/'
    os.chdir(path_flow_root)
    from util.scripts import utils
    params = {}
    with open(utils.fetch_param_file_path(), mode='r') as f:
        params = json.load(f)
    url_for_request_validation = parse.urlunparse((
        params['dgCore']['Scheme'],
        params['dgCore']['Netloc'], 
        'validate',
        '',
        '',
        ''
    ))
    # request validation
    headers = {'content-type': 'application/json'}
    response = requests.post(url_for_request_validation, data=json.dumps(ro_crate), headers=headers)
    result = response.json()
except:
    print('検証サービスに接続できません。')
    print(traceback.format_exc())
else:
    if response.status_code == 200:
        os.chdir(os.environ['HOME'])
        from WORKFLOWS.utils.tmp_validation import tmp_validation
        request_id = result['request_id']
        tmp_validation.save_request_id(request_id)
        print('正常に検証が依頼されました。次の処理にお進みください。\nリクエストID：' + request_id)
    elif response.status_code == 400:
        print('BadRequest')
        print(result['message'])
    elif response.status_code == 500:
        print('InternalServerError')
        print(result['message'])

## 3. 検証結果を確認する

検証サービスから検証結果を取得し、結果を出力します。

In [ ]:
import os
import json
import requests
from urllib import parse
import time
from IPython.display import display, HTML, clear_output
from WORKFLOWS.utils.tmp_validation import tmp_validation
import traceback

request_id = tmp_validation.get_request_id()
counter = 5

while counter >0:
    try:
        # generate url_for_get_validation_results
        path_flow_root = '/home/jovyan/WORKFLOWS/FLOW/'
        os.chdir(path_flow_root)
        from util.scripts import utils
        params = {}
        with open(utils.fetch_param_file_path(), mode='r') as f:
            params = json.load(f)
        url_for_get_validation_results = parse.urlunparse((
            params['dgCore']['Scheme'],
            params['dgCore']['Netloc'], 
            request_id,
            '',
            '',
            ''
        ))
        # request get validation results
        response = requests.get(url_for_get_validation_results)
        result = response.json()
        counter -= 1
        clear_output()
    except:
        print('検証サービスに接続できません。')
        print(traceback.format_exc())
        break
    else:
        if response.status_code == 200:
            status = result['status']
            if status == 'UNKNOWN':
                print('リクエストID：' + request_id + 'の状況が読み込めませんでした。')
                break
            elif any([status == 'QUEUED', status == 'RUNNING']):
                print('リクエストID：' + request_id + 'は検証完了していません。時間をおいて再度確認します。')
                time.sleep(5)
                continue
            elif status == 'COMPLETE':
                tmp_validation.save_verification_results(result)
                print('すべてのメタデータは適切に管理されています。次にお進みください。')
                break
            elif status == 'FAILED':
                tmp_validation.save_verification_results(result)
                result_file_path = tmp_validation.fetch_validation_results_file_path()
                print('メタデータに不備があります。以下のリンクから検証結果を確認し、次にお進みください。')
                display(HTML('<a href=../../../../edit/' + result_file_path + '>検証結果を確認する</a>'))
                break
            elif status == 'EXECUTOR_ERROR':
                print('正常に検証を実行できませんでした。')
                break
            elif status == 'CANCELING':
                print('リクエストID：' + request_id + 'は現在キャンセル中です。')
                break
            elif status == 'CANCELED':
                print('リクエストID：' + request_id + 'はキャンセルされました。')
                break

        elif response.status_code == 400:
            print('BadRequest')
            print(result['message'])
            break
        elif response.status_code == 500:
            print('InternalServerError')
            print(result['message'])
            break
                      
else:
    clear_output()
    print('検証に時間がかかっています。時間をおいて再度このセルを実行してください。')
    raise Exception

## 4. 研究リポジトリに同期する

このタスクの実行結果を研究リポジトリに同期します。  
検証結果を同期するか破棄するかは、「4.1. 検証結果を研究リポジトリに同期するか選択する」で選択できます。

### 4.1. 検証結果を研究リポジトリに同期するか破棄するかを選択する

In [ ]:
import panel as pn

pn.extension()
column = pn.Column()

def save_selection_result(event):
    global need_sync
    done_button.button_type = "success"
    done_button.name = "選択完了しました。次の処理にお進みください。"
    need_sync = True if select.value == 1 else False

select = pn.widgets.Select(name='検証結果を同期するか破棄するかを選択した後、完了ボタンをクリックしてください。', options={'同期する': 1, '同期せずに破棄する':2})
done_button = pn.widgets.Button(name= "選択を完了する", button_type= "primary")
done_button.on_click(save_selection_result)
column.append(select)
column.append(done_button)
column

### 4.2. 研究リポジトリに同期する

研究リポジトリにこのタスクの実行結果を同期します。  
「4.1. 検証結果を研究リポジトリに同期するか破棄するかを選択する」で同期せずに破棄するを選択した場合は、検証結果は同期されずこのファイルの実行結果のみが同期されます。

In [ ]:
import os
from IPython.display import display, Javascript
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils
os.chdir(os.environ['HOME'])
from WORKFLOWS.utils.tmp_validation import tmp_validation

tmp_validation.operate_validation_results(need_sync)
git_path = ['/home/jovyan/WORKFLOWS/FLOW/02_experimental_phase/base_validate_metadata.ipynb']
if need_sync == True:
    validation_result_path = tmp_validation.fetch_validation_result_path()
    git_path.append(validation_result_path)

display(Javascript('IPython.notebook.save_checkpoint();'))
utils.syncs_with_repo(git_path=git_path, gitannex_path=None, gitannex_files=None, message='メタデータ検証')

## 5. 研究ワークフロー機能トップページに遷移する

以下のセルを実行し、表示されるリンクをクリックして研究ワークフロー機能トップページに戻ってください。

In [ ]:
from IPython.display import display, HTML, Javascript
display(HTML("<a href='../../base_FLOW.ipynb'>研究ワークフロー機能トップページに遷移する</a>"))
display(Javascript('IPython.notebook.save_checkpoint();'))